***Substitution cipher***

In [1]:
default_cipher_alphabet = ['p', 'h', 'q', 'g', 'i', 'u', 'm', 'e', 'a', 'y', 'l', 'n', 'o', 'f', 'd', 'x',
                           'j', 'k', 'r', 'c', 'v', 's', 't', 'z', 'w', 'b']
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 
                't', 'u', 'v', 'w', 'x', 'y', 'z']


def substitution_cipher_encrypt(string: str, cipher_alphabet: list = default_cipher_alphabet):
    cipher_dict = dict(zip(alphabet, cipher_alphabet))
    cs = []
    for i in range(len(string)):
        if string[i].lower() in cipher_dict:
            cs.append(cipher_dict[string[i]] if string[i].islower() else cipher_dict[string[i].lower()].upper())
        else:
            cs.append(string[i])
        
    return ''.join(cs)


def substitution_cipher_decrypt(string: str, cipher_alphabet: list = default_cipher_alphabet):
    cipher_dict = dict(zip(cipher_alphabet, alphabet))
    cs = []
    for i in range(len(string)):
        if string[i].lower() in cipher_dict:
            cs.append(cipher_dict[string[i]] if string[i].islower() else cipher_dict[string[i].lower()].upper())
        else:
            cs.append(string[i])
        
    return ''.join(cs)

# Example 1
text = 'Substitutional cipher!'
encrypted_text = substitution_cipher_encrypt(text)
print('Substitutional cipher')
print(f'Example 1:')
print(f'\t\tOriginal text: {text}')
print(f'\t\tEncrypted Text: {encrypted_text}')
print(f'\t\tDecrypted Text: {substitution_cipher_decrypt(encrypted_text)}')
# Example 2
text = 'Yes it\'s working'
encrypted_text = substitution_cipher_encrypt(text)
print(f'Example 2:')
print(f'\t\tOriginal text: {text}')
print(f'\t\tEncrypted Text: {encrypted_text}')
print(f'\t\tDecrypted Text: {substitution_cipher_decrypt(encrypted_text)}')

Substitutional cipher
Example 1:
		Original text: Substitutional cipher!
		Encrypted Text: Rvhrcacvcadfpn qaxeik!
		Decrypted Text: Substitutional cipher!
Example 2:
		Original text: Yes it's working
		Encrypted Text: Wir ac'r tdklafm
		Decrypted Text: Yes it's working


***Affine Cipher***

In [2]:
import re


# Extended Euclidean Algorithm
def egcd(a: int, b: int):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b // a, b % a
        m, n = x - u * q, y - v * q
        b,a, x,y, u,v = a,r, u,v, m,n
    gcd = b

    return gcd, x, y


def modinv(a: int, m: int):
    gcd, x, y = egcd(a, m)
    if gcd != 1:
        return None
    else:
        return x % m


def affine_encrypt(string: str, keys: list = [23, 30]):
    """
    C = (a*P + b) % 26
    """
    res = ''
    for c in string:
        if re.match('\W', c):
            res += c
        elif c.isupper():
            res += chr(((keys[0] * (ord(c.lower()) - ord('a')) + keys[1]) % 26) + ord('a')).upper()
        else:
            res += chr(((keys[0] * (ord(c) - ord('a')) + keys[1]) % 26) + ord('a'))
    
    return res


def affine_decrypt(string: str, keys: list = [23, 30]):
    """
    P = (a^-1 * (C - b)) % 26
    """
    res = ''
    for c in string:
        if re.match('\W', c):
            res += c
        elif c.isupper():
            res += chr(((modinv(keys[0], 26) * (ord(c.lower()) - ord('a') - keys[1])) % 26) + ord('a')).upper()
        else:
            res += chr(((modinv(keys[0], 26) * (ord(c) - ord('a') - keys[1])) % 26) + ord('a'))
    
    return res


# Example 1
text = 'string'
encrypted_text = affine_encrypt(text)
print('Affine cipher')
print(f'Example 1:')
print(f'\t\tOriginal text: {text}')
print(f'\t\tEncrypted Text: {encrypted_text}')
print(f'\t\tDecrypted Text: {affine_decrypt(encrypted_text)}')
# Example 2
text = 'Yes it\'s working'
encrypted_text = affine_encrypt(text)
print(f'Example 2:')
print(f'\t\tOriginal text: {text}')
print(f'\t\tEncrypted Text: {encrypted_text}')
print(f'\t\tDecrypted Text: {affine_decrypt(encrypted_text)}')

Affine cipher
Example 1:
		Original text: string
		Encrypted Text: czfgrm
		Decrypted Text: string
Example 2:
		Original text: Yes it's working
		Encrypted Text: Ksc gz'c qofagrm
		Decrypted Text: Yes it's working


***Affine recurrent cipher*** <br/>
Для этого необходимо задать две ключевые пары k1 = (α1, β1), k2 = (α2, β2), и тогда ключевая пара для произвольного символа преобразуемой последовательности будет иметь вид ki = (αi−1αi−2, βi−1 + βi−2), i = 3, l.

In [3]:
def get_keys(keys_last: list, keys_prev: list):
    return [keys_last[0] * keys_prev[0], keys_last[1] * keys_prev[1]]


def affine_recurrent_encrypt(string: str, keys1: list = [17, 30], keys2: list = [23, 30]):
    res = ''
    keys_last = keys2
    keys_prev = keys1
    keys = get_keys(keys_last, keys_prev)
    for i in range(len(string)):
        if i >= 2:
            res += affine_encrypt(string[i], keys)
            keys_prev = keys_last
            keys_last = keys
            keys = get_keys(keys_last, keys_prev)
        elif i == 1:
            res += affine_encrypt(string[i], keys2)
        elif i == 0:
            res += affine_encrypt(string[i], keys1)

    return res


def affine_recurrent_decrypt(string: str, keys1: list = [17, 30], keys2: list = [23, 30]):
    res = ''
    keys_last = keys2
    keys_prev = keys1
    keys = get_keys(keys_last, keys_prev)
    for i in range(len(string)):
        if i >= 2:
            res += affine_decrypt(string[i], keys)
            keys_prev = keys_last
            keys_last = keys
            keys = get_keys(keys_last, keys_prev)
        elif i == 1:
            res += affine_decrypt(string[i], keys2)
        elif i == 0:
            res += affine_decrypt(string[i], keys1)

    return res


# Example 1
text = 'Affine cipher!'
encrypted_text = affine_recurrent_encrypt(text)
print('Affine cipher')
print(f'Example 1:')
print(f'\t\tOriginal text: {text}')
print(f'\t\tEncrypted Text: {encrypted_text}')
print(f'\t\tDecrypted Text: {affine_recurrent_decrypt(encrypted_text)}')
# Example 2
text = 'Yes it\'s working'
encrypted_text = affine_recurrent_encrypt(text)
print(f'Example 2:')
print(f'\t\tOriginal text: {text}')
print(f'\t\tEncrypted Text: {encrypted_text}')
print(f'\t\tDecrypted Text: {affine_recurrent_decrypt(encrypted_text)}')

Affine cipher
Example 1:
		Original text: Affine cipher!
		Encrypted Text: Epvoxa uqldal!
		Decrypted Text: Affine cipher!
Example 2:
		Original text: Yes it's working
		Encrypted Text: Wsi mf'g qwngqjk
		Decrypted Text: Yes it's working


In [4]:
isExit = False
while not isExit:
    print('Choose cipher alghoritm:')
    cipher = input('1 - substitution, 2 - affine, 3 - affine reccurent alghoritm\nYour choice: ')
    text = input('Input text to encrypt: ')
    encrypted = ''
    decrypted = ''
    if cipher == '1':
        choice = input('Do you want to input your own alphabet to encrypt? (y/n)\n')
        _alphabet = input('Input your custom alphabet (26 characters without spaces): ').split('') if choice == 'y' else default_cipher_alphabet 
        encrypted = substitution_cipher_encrypt(text, _alphabet)
        decrypted = substitution_cipher_decrypt(encrypted, _alphabet)
    elif cipher == '2':
        choice = input('Do you want to use custom affine cipher keys? (y/n)\n')
        keys = list(map(int, input('Input your custom affine keys (2 numbers splited with ", "): ').split(', '))) if choice == 'y' else [17, 30]
        encrypted = affine_encrypt(text, keys)
        decrypted = affine_decrypt(encrypted, keys)
    elif cipher == '3':
        choice = input('Do you want to use custom affine recurrent cipher keys? (y/n)\n')
        print('Both affine recurrent key sets are 2 numbers splited with ", "')
        keys1 = list(map(int, input('Input first key set: ').split(', '))) if choice == 'y' else [17, 30]
        keys2 = list(map(int, input('Input second key set: ').split(', '))) if choice == 'y' else [23, 30]
        encrypted = affine_recurrent_encrypt(text, keys1, keys2)
        decrypted = affine_recurrent_decrypt(encrypted, keys1, keys2)
    else:
        print('No such cipher')
    print(f'Original text: {text}')
    print(f'Encrypted text: {encrypted}')
    print(f'Decrypted text: {decrypted}')
    isExit = False if input('Do you want to continue (y/n)\n') == 'y' else True

Choose cipher alghoritm:
1 - substitution, 2 - affine, 3 - affine reccurent alghoritm
Your choice:  1
Input text to encrypt:  I was made for lovin you baby
Do you want to input your own alphabet to encrypt? (y/n)
 n
Original text: I was made for lovin you baby
Encrypted text: A tpr opgi udk ndsaf wdv hphw
Decrypted text: I was made for lovin you baby
Do you want to continue (y/n)
 n


In [5]:
l = [q - 97 for q in [115, 116, 114, 105, 110, 103]]
n = [(a * 23 + 30) % 26 + 97 for a in l]
n

[99, 122, 102, 103, 114, 109]

In [11]:
modinv(23, 26)

17